In [1]:
# let us first import the libraries
# local import for convinience
import sys, os
sys.path.insert(1, '../src/SupplyNetPy/Components/')
import core as scm
import utilities as scm

import simpy
import numpy as np
import matplotlib.pyplot as plt


def print_node_wise_performance(nodes_object_list):
    if not nodes_object_list:
        print("No nodes provided.")
        return

    # Pre-fetch statistics from all nodes
    stats_per_node = {node.name: node.stats.get_statistics() for node in nodes_object_list}
    stat_keys = sorted(next(iter(stats_per_node.values())).keys())

    # Determine column widths
    col_width = 25
    header = "Performance Metric".ljust(col_width)
    for name in stats_per_node:
        header += name.ljust(col_width)
    print(header)

    # Print row-wise stats
    for key in stat_keys:
        row = key.ljust(col_width)
        for name in stats_per_node:
            value = stats_per_node[name].get(key, "N/A")
            row += str(value).ljust(col_width)
        print(row)

def print_sc_performance(scnet):
    performance_keys = sorted([
        "available_inv", "avg_available_inv", "inventory_carry_cost", "inventory_spend_cost",
        "transportation_cost", "revenue", "total_cost", "profit", "demand_by_customers",
        "fulfillment_received_by_customers", "demand_by_site", "fulfillment_received_by_site",
        "total_demand", "total_fulfillment_received", "shortage", "backorders",
        "avg_cost_per_order", "avg_cost_per_item"
    ])

    print("\n--- Supply Chain Performance Summary ---\n")
    max_key_length = max(len(key) for key in performance_keys) + 2
    for key in performance_keys:
        value = scnet.get(key, "N/A")
        print(f"{key.ljust(max_key_length)}: {value}")

In [2]:
"""
Testing replenishment policies with a simple supply chain network
"""
import simpy
env = simpy.Environment()

supplier = scm.Supplier(env=env, ID='S1', name='Supplier 1', node_type='infinite_supplier',logging=False)

distributor = scm.InventoryNode(env=env, ID='D1', name='Distributor1', node_type='distributor', 
                                capacity=float('inf'), initial_level=5, inventory_holding_cost=0.1, 
                                replenishment_policy=scm.SSReplenishment, logging=True,
                                policy_param={'s':5,'S':11}, product_sell_price=100, product_buy_price=90)

link1  = scm.Link(env=env, ID='L1', source=supplier, sink=distributor, cost=10, lead_time=lambda: 2)

demand1 = scm.Demand(env=env, ID='d1', name='Demand1', order_arrival_model=lambda: 0.3, order_quantity_model=lambda:1,
                     logging=True,demand_node=distributor, delivery_cost=lambda: 10, lead_time=lambda: 2)

demand2 = scm.Demand(env=env, ID='d2', name='Demand2', order_arrival_model=lambda: 0.4, order_quantity_model=lambda:1, 
                     logging=True,demand_node=distributor, delivery_cost=lambda: 10, lead_time=lambda: 2)

supplynet = scm.create_sc_net(env=env,nodes=[supplier,distributor],links=[link1],demands=[demand1, demand2])
supplynet = scm.simulate_sc_net(supplynet, sim_time=9, logging=True)
inv_levels = np.array(distributor.inventory.instantaneous_levels)
#plt.plot(inv_levels[:,0], inv_levels[:,1], label='Inventory Level at D1', marker='.', color='blue')
#plt.grid()
print_node_wise_performance([distributor, demand1, demand2])
#print_sc_performance(supplynet)
# check time t=4.0 - d2 c11 came before inventory was replenished
# check time t=8.4 - d1 c29 consumed 1 unit, inventory dropped to 5, replenishment order placed for 6, d2 c22 consumed 1, inv dropped to 4

INFO D1 - 0.0000:D1: Inventory levels:5, on hand:5
INFO D1 - 0.0000:D1:Replenishing inventory from supplier:Supplier 1, order placed for 6 units.
INFO D1 - 0.0000:D1:shipment in transit from supplier:Supplier 1.
INFO d1 - 0.0000:d1:Customer1:Order quantity:1, available.
INFO D1 - 0.0000:D1: Inventory levels:3, on hand:9
INFO d2 - 0.0000:d2:Customer1:Order quantity:1, available.
INFO d1 - 0.3000:d1:Customer2:Order quantity:1, available.
INFO D1 - 0.3000:D1: Inventory levels:2, on hand:8
INFO d2 - 0.4000:d2:Customer2:Order quantity:1, available.
INFO D1 - 0.4000:D1: Inventory levels:1, on hand:7
INFO d1 - 0.6000:d1:Customer3:Order quantity:1, available.
INFO D1 - 0.6000:D1: Inventory levels:0, on hand:6
INFO d2 - 0.8000:d2:Customer3: Order quantity:1 not available, inventory level:0. No tolerance! Shortage:1.
INFO d1 - 0.9000:d1:Customer4: Order quantity:1 not available, inventory level:0. No tolerance! Shortage:1.
INFO d1 - 1.2000:d1:Customer5: Order quantity:1 not available, inventory 

INFO D1 - 2.0000:D1:shipment in transit from supplier:Supplier 1.
INFO d1 - 2.1000:d1:Customer8:Order quantity:1, available.
INFO D1 - 2.1000:D1: Inventory levels:4, on hand:10
INFO d2 - 2.4000:d2:Customer7:Order quantity:1, available.
INFO D1 - 2.4000:D1: Inventory levels:2, on hand:8
INFO d1 - 2.4000:d1:Customer9:Order quantity:1, available.
INFO d1 - 2.7000:d1:Customer10:Order quantity:1, available.
INFO D1 - 2.7000:D1: Inventory levels:1, on hand:7
INFO d2 - 2.8000:d2:Customer8:Order quantity:1, available.
INFO D1 - 2.8000:D1: Inventory levels:0, on hand:6
INFO d1 - 3.0000:d1:Customer11: Order quantity:1 not available, inventory level:0. No tolerance! Shortage:1.
INFO d2 - 3.2000:d2:Customer9: Order quantity:1 not available, inventory level:0. No tolerance! Shortage:1.
INFO d1 - 3.3000:d1:Customer12: Order quantity:1 not available, inventory level:0. No tolerance! Shortage:1.
INFO d1 - 3.6000:d1:Customer13: Order quantity:1 not available, inventory level:0. No tolerance! Shortage:1

Performance Metric       Distributor1             Demand1                  Demand2                  
backorder                [0, 0]                   [0, 0]                   [0, 0]                   
demand_fulfilled         [22, 22]                 [0, 0]                   [0, 0]                   
demand_placed            [5, 30]                  [30, 30]                 [23, 23]                 
demand_received          [27, 27]                 [0, 0]                   [0, 0]                   
fulfillment_received     [4, 24]                  [13, 13]                 [9, 9]                   
inventory_carry_cost     1.3000000000000007       0                        0                        
inventory_level          2                        0                        0                        
inventory_spend_cost     2160                     0                        0                        
inventory_waste          0                        0                        0               

In [35]:
import simpy

def test_process(env):
    while True:
        print(f"eve A: {env.now}")
        yield env.timeout(0.4)

def test_process2(env):
    while True:
        print(f"eve B: {env.now}")
        yield env.timeout(0.3)

env = simpy.Environment()
env.process(test_process(env))
env.process(test_process2(env))
env.run(until=2.400001)
# This might be because of how Python stores floats and handles precision.

eve A: 0
eve B: 0
eve B: 0.3
eve A: 0.4
eve B: 0.6
eve A: 0.8
eve B: 0.8999999999999999
eve B: 1.2
eve A: 1.2000000000000002
eve B: 1.5
eve A: 1.6
eve B: 1.8
eve A: 2.0
eve B: 2.1
eve A: 2.4
eve B: 2.4


In [ ]:
"""
Testing replenishment policies with a simple supply chain network
"""
import simpy
env = simpy.Environment()

supplier = scm.Supplier(env=env, ID='S1', name='Supplier 1', node_type='infinite_supplier',logging=False)

distributor = scm.InventoryNode(env=env, ID='D1', name='Distributor1', node_type='distributor', 
                                capacity=float('inf'), initial_level=5, inventory_holding_cost=0.1, 
                                replenishment_policy=scm.PeriodicReplenishment, logging=True,
                                policy_param={'T':1,'Q':3}, product_sell_price=100, product_buy_price=90)

link1  = scm.Link(env=env, ID='L1', source=supplier, sink=distributor, cost=10, lead_time=lambda: 2)

demand1 = scm.Demand(env=env, ID='d1', name='Demand1', order_arrival_model=lambda: 0.3, order_quantity_model=lambda:1,
                     logging=True,demand_node=distributor, delivery_cost=lambda: 10, lead_time=lambda: 2)

#demand2 = scm.Demand(env=env, ID='d2', name='Demand2', order_arrival_model=lambda: 0.4, order_quantity_model=lambda:1, 
#                     logging=True,demand_node=distributor, delivery_cost=lambda: 10, lead_time=lambda: 2)

supplynet = scm.create_sc_net(env=env,nodes=[supplier,distributor],links=[link1],demands=[demand1])
supplynet = scm.simulate_sc_net(supplynet, sim_time=10, logging=True)
inv_levels = np.array(distributor.inventory.instantaneous_levels)
#plt.plot(inv_levels[:,0], inv_levels[:,1], label='Inventory Level at D1', marker='.', color='blue')
#plt.grid()
print_node_wise_performance([distributor, demand1])

# carry cost mismatch with AnyLogistix
# till t=9 it is correct = 1.32
# For us demand at 9.00 is fulfilled, but in AL demand at 9.9 is fulfilled
# 
# at t=3.00, t=6.00, the demand d1 arrives first (and inv not available), then inventory is replenished. 
# But at 9.00 inventory is replenished first and then demand d1 comes.

INFO D1 - 0.0000:D1: Inventory levels:5, on hand:5
INFO D1 - 0.0000:D1:Replenishing inventory from supplier:Supplier 1, order placed for 3 units.
INFO D1 - 0.0000:D1:shipment in transit from supplier:Supplier 1.
INFO d1 - 0.0000:d1:Customer1:Order quantity:1, available.
INFO d1 - 0.3000:d1:Customer2:Order quantity:1, available.
INFO d1 - 0.6000:d1:Customer3:Order quantity:1, available.
INFO d1 - 0.9000:d1:Customer4:Order quantity:1, available.
INFO D1 - 1.0000:D1: Inventory levels:1, on hand:4
INFO D1 - 1.0000:D1:Replenishing inventory from supplier:Supplier 1, order placed for 3 units.
INFO D1 - 1.0000:D1:shipment in transit from supplier:Supplier 1.
INFO d1 - 1.2000:d1:Customer5:Order quantity:1, available.
INFO d1 - 1.5000:d1:Customer6: Order quantity:1 not available, inventory level:0. No tolerance! Shortage:1.
INFO d1 - 1.8000:d1:Customer7: Order quantity:1 not available, inventory level:0. No tolerance! Shortage:1.
INFO D1 - 2.0000:D1:Inventory replenished. reorder_quantity=3, In

Performance Metric       Distributor1             Demand1                  
backorder                [0, 0]                   [0, 0]                   
demand_fulfilled         [23, 23]                 [0, 0]                   
demand_placed            [10, 30]                 [34, 34]                 
demand_received          [29, 29]                 [0, 0]                   
fulfillment_received     [8, 24]                  [23, 23]                 
inventory_carry_cost     1.4099999999999975       0                        
inventory_level          0                        0                        
inventory_spend_cost     2160                     0                        
inventory_waste          0                        0                        
node_cost                2261.41                  290                      
orders_shortage          [5, 5]                   [0, 0]                   
profit                   638.5900000000001        -290                     
revenue     

In [2]:
"""
Testing Back order allowed/not allowed for replenishment policies (interger values)
"""
env = simpy.Environment()

supplier = scm.Supplier(env=env, ID='S1', name='Supplier 1', node_type='infinite_supplier',logging=False)



distributor = scm.InventoryNode(env=env, ID='D1', name='Distributor1', node_type='distributor', 
                                capacity=float('inf'), initial_level=0, inventory_holding_cost=0.1, 
                                replenishment_policy=scm.RQReplenishment, logging=True,
                                policy_param={'R':0,'Q':1}, product_sell_price=100, product_buy_price=90)

link1  = scm.Link(env=env, ID='L1', source=supplier, sink=distributor, cost=10, lead_time=lambda: 2)

demand1 = scm.Demand(env=env, ID='d1', name='Demand1', order_arrival_model=lambda: 2, order_quantity_model=lambda:1,
                     logging=True,demand_node=distributor, delivery_cost=lambda: 10, lead_time=lambda: 0)

supplynet = scm.create_sc_net(env=env,nodes=[supplier,distributor],links=[link1],demands=[demand1])
supplynet = scm.simulate_sc_net(supplynet, sim_time=10, logging=True)
inv_levels = np.array(distributor.inventory.instantaneous_levels)
print_node_wise_performance([distributor, demand1])

INFO D1 - 0.0000:D1: Inventory levels: 0, on hand: 0
INFO D1 - 0.0000:D1:Replenishing inventory from supplier:Supplier 1, order placed for 1 units.
INFO d1 - 0.0000:d1:Customer1: Order quantity:1 not available, inventory level:0. No tolerance! Shortage:1.
INFO D1 - 0.0000:D1:shipment in transit from supplier:Supplier 1.
INFO d1 - 2.0000:d1:Customer2: Order quantity:1 not available, inventory level:0. No tolerance! Shortage:1.
INFO D1 - 2.0000:D1:Inventory replenished. reorder_quantity=1, Inventory levels:1
INFO d1 - 4.0000:d1:Customer3:Order quantity:1, available.
INFO D1 - 4.0000:D1: Inventory levels: 0, on hand: 0
INFO D1 - 4.0000:D1:Replenishing inventory from supplier:Supplier 1, order placed for 1 units.
INFO D1 - 4.0000:D1:shipment in transit from supplier:Supplier 1.
INFO d1 - 6.0000:d1:Customer4: Order quantity:1 not available, inventory level:0. No tolerance! Shortage:1.
INFO D1 - 6.0000:D1:Inventory replenished. reorder_quantity=1, Inventory levels:1
INFO d1 - 8.0000:d1:Custo

Performance Metric       Distributor1             Demand1                  
backorder                [0, 0]                   [0, 0]                   
demand_fulfilled         [2, 2]                   [0, 0]                   
demand_placed            [3, 3]                   [5, 5]                   
demand_received          [2, 2]                   [0, 0]                   
fulfillment_received     [2, 2]                   [2, 2]                   
inventory_carry_cost     0.4                      0                        
inventory_level          0                        0                        
inventory_spend_cost     180                      0                        
inventory_waste          0                        0                        
node_cost                210.4                    20                       
orders_shortage          [3, 3]                   [0, 0]                   
profit                   -10.400000000000006      -20                      
revenue     

In [6]:
env = simpy.Environment()

inventory2 = simpy.Container(env, capacity=float('inf'), init=float('inf'))
inventory = simpy.Container(env, capacity=float('inf'), init=0)
inv_drop = env.event()

def demand(env, inventory):
    """
    Demand of 1 unit every 2 days, starts at day 0
    Consume if available, if not available, leave immidiately.
    """
    global inv_drop
    while True:
        #yield env.timeout(0.0)
        print(f"{env.now}:B: Demand for 1 ")
        if inventory.level >= 1:
            yield inventory.get(1.0)
            inv_drop.succeed()
            inv_drop = env.event()
            print(f"{env.now}:B: Fulfilled, Inventory: {inventory.level}")
        else:
            print(f"{env.now}:B: not fulfilled at, Inventory: {inventory.level}")
        yield env.timeout(2.0)

def replenish(env, inventory):
    """
    Instanteneous replenishment
    Reorder quantity is always 1 unit, lead time is 2 
    Order when inventory level < 1
    (R,Q) policy with R=0, Q=1
    """
    global inv_drop
    while True:
        replenished_amount = 1  # Fixed replenishment amount
        if(inventory.level < 1):
            print(f"{env.now}:A: Ordering replenishment for {replenished_amount} units")
            #yield inventory2.get(replenished_amount) # get it from the supplier inventory2
            yield env.timeout(2.0)  # Replenishment every 2 time units
            inventory.put(replenished_amount)
            print(f"{env.now}:A: Shipment received. Inventory replenished to {inventory.level}")
        else:
            yield inv_drop

replenish_event = env.process(replenish(env, inventory)) 
demand_event = env.process(demand(env, inventory))

env.run(until=10)
print(f"Inventory lvl: {inventory.level}")

# When process 'replenish' is created before 'demand', it follows the order for subsequent events created,
# and the inventory is replenished before demand is processed. (This is observed in AnyLogistix)
# Example 1: 
# 0: replenishment order placed
# 0: demand comes
# 2: replenishment received
# 2: demand comes

# Example 2:
# 0: demand comes (since sufficient inv lvls, no replenishment order is placed, thats why demand comes first)
# 0: replenishment order placed (since after demand is satisfied, inventory lvl drops)
# 2: demand comes 
# 2: replenishment received

# When demand is created before replenish, the demand is processed first, and then the inventory is replenished
# which may lead to unsatisfied demand (This is not correct order)

# Another scenario: when 'yield inventory2.get()' is used, causes the 'replenish' process to fall behind the 
# 'demand' process, leading to unsatisfied demand


0:A: Ordering replenishment for 1 units
0:B: Demand for 1 
0:B: not fulfilled at, Inventory: 0
2.0:A: Shipment received. Inventory replenished to 1
2.0:B: Demand for 1 
2.0:B: Fulfilled, Inventory: 0.0
2.0:A: Ordering replenishment for 1 units
4.0:B: Demand for 1 
4.0:B: not fulfilled at, Inventory: 0.0
4.0:A: Shipment received. Inventory replenished to 1.0
6.0:B: Demand for 1 
6.0:B: Fulfilled, Inventory: 0.0
6.0:A: Ordering replenishment for 1 units
8.0:B: Demand for 1 
8.0:B: not fulfilled at, Inventory: 0.0
8.0:A: Shipment received. Inventory replenished to 1.0
Inventory lvl: 1.0


In [ ]:
"""
Testing for bugs and errors
"""

import simpy
env = simpy.Environment()
raw_mat = scm.RawMaterial(ID="rm1", name="raw1", extraction_quantity=30, extraction_time=1, mining_cost=0.1,cost=1)
supplier = scm.Supplier(env=env, ID='S1', name='Supplier1', node_type='supplier',capacity=2000,inventory_holding_cost=0.02,
                        raw_material=raw_mat,logging=False)
product = scm.Product(ID='P1', name='Product1', manufacturing_cost=20, manufacturing_time=1,sell_price=50,
                      raw_materials = [(raw_mat,1)], batch_size=1)
factory = scm.Manufacturer(env=env, ID='F1', name='Factory1', capacity=20, initial_level=5, inventory_holding_cost=0.1,
                           product_sell_price=50, replenishment_policy=scm.SSReplenishment, policy_param={'s':5,'S':20},
                           product =product, logging=False)
distributor = scm.InventoryNode(env=env, ID='D1', name='Distributor1', node_type='distributor', capacity=15, initial_level=10, 
                                inventory_holding_cost=1, replenishment_policy=scm.SSReplenishment, logging=True,
                                policy_param={'s':5,'S':15,'period':1}, 
                                product_sell_price=52, product_buy_price=35)
link1 = scm.Link(env=env, ID='L1', source=supplier, sink=factory, cost=10, lead_time=lambda: 2)
link2 = scm.Link(env=env, ID='L2', source=factory, sink=distributor, cost=5, lead_time=lambda: 1)
demand2 = scm.Demand(env=env, ID='d2', name='Demand2', order_arrival_model=lambda: 1, order_quantity_model=lambda:1, 
                     demand_node=distributor, delivery_cost=lambda: 10, lead_time=lambda: 2, logging=True)

scm.global_logger.enable_logging()
import numpy as np
import matplotlib.pyplot as plt
supplynet = scm.create_sc_net(env=env,nodes=[supplier,factory,distributor],links=[link1],demands=[demand2])
supplynet = scm.simulate_sc_net(supplynet, sim_time=31)
inv_levels = np.array(distributor.inventory.instantaneous_levels)
plt.plot(inv_levels[:,0], inv_levels[:,1], label='Inventory Level at D1', marker='.', color='blue')
plt.grid()
print_sc_performance(supplynet)

In [ ]:
"""
Testing supplier selection policies with multiple suppliers
"""

import simpy
env = simpy.Environment()
supplier = scm.Supplier(env=env, ID='S1', name='Supplier 1', node_type='infinite_supplier',logging=False)
supplier2 = scm.Supplier(env=env, ID='S2', name='Supplier 2', node_type='infinite_supplier',logging=False)
supplier3 = scm.Supplier(env=env, ID='S3', name='Supplier 3', node_type='infinite_supplier',logging=False)
distributor = scm.InventoryNode(env=env, ID='D1', name='Distributor 1', node_type='distributor', capacity=500, initial_level=500, 
                                inventory_holding_cost=0.2, replenishment_policy=scm.SSReplenishment, policy_param={'s':200,'S':500}, 
                                product_sell_price=250, product_buy_price=200, supplier_selection_policy=scm.SelectFirst)
link1  = scm.Link(env=env, ID='L1', source=supplier, sink=distributor, cost=10, lead_time=lambda: 3)
link2  = scm.Link(env=env, ID='L2', source=supplier2, sink=distributor, cost=20, lead_time=lambda: 2)
link3  = scm.Link(env=env, ID='L3', source=supplier3, sink=distributor, cost=30, lead_time=lambda: 1)
demand1 = scm.Demand(env=env, ID='d1', name='Demand 1', order_arrival_model=lambda: 2, order_quantity_model=lambda:80, 
                     demand_node=distributor, delivery_cost=lambda: 10, lead_time=lambda: 1,logging=False)
demand2 = scm.Demand(env=env, ID='d2', name='Demand 2', order_arrival_model=lambda: 2, order_quantity_model=lambda:120, 
                     demand_node=distributor, delivery_cost=lambda: 10, lead_time=lambda: 1,logging=False)
scm.global_logger.enable_logging()

import numpy as np
import matplotlib.pyplot as plt
supplynet = scm.create_sc_net(env=env, nodes=[supplier, supplier2, supplier3, distributor],links=[link1, link2, link3],demands=[demand1,demand2])
supplynet = scm.simulate_sc_net(supplynet, sim_time=10)
inv_levels = np.array(distributor.inventory.instantaneous_levels)
plt.plot(inv_levels[:,0], inv_levels[:,1], label='Inventory Level at D1', marker='.', color='blue')
plt.grid()
print_sc_performance(supplynet)